In [5]:
import pandas as pd
import numpy as np
dt = pd.read_pickle('Loan_binning.pickle')
dt

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,APPLICANTINCOME_BIN,COAPPLICANTINCOME_BIN,LOANAMOUNT_BIN,LOAN_AMOUNT_TERM_BIN
2,1,2,0,1,0,3000,0.0,66.0,360.0,1.0,1,1,"(2589.8, 3031.3]","(-0.001, 1040.5]","(16.999, 74.7]","(300.0, 360.0]"
3,1,2,0,0,1,2583,2358.0,120.0,360.0,1.0,1,1,"(2232.2, 2589.8]","(2083.0, 2500.0]","(110.1, 120.0]","(300.0, 360.0]"
4,1,1,0,1,1,6000,0.0,141.0,360.0,1.0,1,1,"(5417.0, 6460.0]","(-0.001, 1040.5]","(138.2, 158.0]","(300.0, 360.0]"
5,1,2,1,1,0,5417,4196.0,267.0,360.0,1.0,1,1,"(4435.8, 5417.0]","(3831.2, 20000.0]","(235.5, 700.0]","(300.0, 360.0]"
6,1,2,0,0,1,2333,1516.0,95.0,360.0,1.0,1,1,"(2232.2, 2589.8]","(1040.5, 1702.0]","(74.7, 96.0]","(300.0, 360.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,1,2,0,1,1,3597,2157.0,119.0,360.0,0.0,2,0,"(3418.0, 3887.5]","(2083.0, 2500.0]","(110.1, 120.0]","(300.0, 360.0]"
495,0,2,1,1,1,3326,913.0,105.0,84.0,1.0,3,1,"(3031.3, 3418.0]","(-0.001, 1040.5]","(96.0, 110.1]","(35.999, 300.0]"
496,1,2,0,0,1,2600,1700.0,107.0,360.0,1.0,2,1,"(2589.8, 3031.3]","(1040.5, 1702.0]","(96.0, 110.1]","(300.0, 360.0]"
498,1,2,1,1,0,2895,0.0,95.0,360.0,1.0,3,1,"(2589.8, 3031.3]","(-0.001, 1040.5]","(74.7, 96.0]","(300.0, 360.0]"


In [6]:
def num_event(event, dataframe ,field, val):
    if event == 'good':
        event = 1
    else:
        event = 0
    cond_main = dataframe[field] == val
    cond_event = dataframe['Loan_Status'] == event
    n = len(dataframe[cond_main & cond_event].index)
    return n


def dist_event(event, dataframe ,field, val):
    n = num_event(event, dataframe, field, val)
    sumd = 0
    all_elements = set(dataframe[field].to_numpy())
    for a in all_elements:
        s = num_event(event, dataframe, field, a)
        sumd = sumd + s
    return n/sumd

def WOE(dataframe, field, val):
    dg = dist_event('good', dataframe, field, val)
    db = dist_event('bad', dataframe, field, val)
    w = np.log(dg / db)
    return w

def IV(dataframe, field):
    elements = set(dataframe[field].to_numpy())
    n = len(elements)
    sumiv = 0
    for i in elements:
        dg = dist_event('good', dataframe, field, i)
        db = dist_event('bad', dataframe, field, i)
        s = (dg-db) * WOE(dataframe, field, i)
        sumiv = sumiv + s
    return sumiv   

fields = ['Gender','Married','Dependents','Education','Self_Employed','Credit_History','Property_Area','APPLICANTINCOME_BIN','COAPPLICANTINCOME_BIN','LOANAMOUNT_BIN','LOAN_AMOUNT_TERM_BIN']
list_woe=[]
list_attr=[]
list_val=[]
list_ngood = []
list_nbad = []
list_dgood = []
list_dbad = []

for f in fields:
    the_set = set(dt[f].to_numpy())
   
    for s in the_set:
        n_good = num_event('good', dt, f, s)
        n_bad = num_event('bad', dt, f, s)
        d_good = dist_event('good', dt, f, s)
        d_bad = dist_event('bad', dt, f, s)
        woe = WOE(dt, f, s)

        if 'BIN' in f:
            s = str(s)
        list_attr.append(f)
        list_val.append(s)
        list_ngood.append(n_good)
        list_nbad.append(n_bad)
        list_dgood.append(d_good)
        list_dbad.append(d_bad)
        list_woe.append(woe)

x = list(zip(list_attr, list_val, list_ngood, list_nbad, list_dgood, list_dbad,list_woe)) 
xlabel = ['ATTR', 'VAL', 'N_GOOD', 'N_BAD', 'DIST_GOOD', 'DIST_BAD','WOE']
df = pd.DataFrame(x, columns = xlabel )

df = df.sort_values(by=['ATTR', 'VAL'])

df = df.reset_index(drop=True)



df

,ATTR,VAL,N_GOOD,N_BAD,DIST_GOOD,DIST_BAD,WOE
0,APPLICANTINCOME_BIN,"(149.999, 2232.2]",23,11,0.100000,0.101852,-0.018349
1,APPLICANTINCOME_BIN,"(2232.2, 2589.8]",26,8,0.113043,0.074074,0.422707
2,APPLICANTINCOME_BIN,"(2589.8, 3031.3]",25,9,0.108696,0.083333,0.265703
3,APPLICANTINCOME_BIN,"(3031.3, 3418.0]",23,11,0.100000,0.101852,-0.018349
4,APPLICANTINCOME_BIN,"(3418.0, 3887.5]",17,16,0.073913,0.148148,-0.695323
5,APPLICANTINCOME_BIN,"(3887.5, 4435.8]",24,10,0.104348,0.092593,0.119521
6,APPLICANTINCOME_BIN,"(4435.8, 5417.0]",23,12,0.100000,0.111111,-0.105361
7,APPLICANTINCOME_BIN,"(5417.0, 6460.0]",23,9,0.100000,0.083333,0.182322
8,APPLICANTINCOME_BIN,"(6460.0, 9883.1]",26,8,0.113043,0.074074,0.422707
9,APPLICANTINCOME_BIN,"(9883.1, 81000.0]",20,14,0.086957,0.129630,-0.399273


In [7]:
df.to_pickle('Loan_attr.pickle')

In [8]:
df_iv = pd.DataFrame({'ATTR': [], 'IV': []})
for f in fields:
    d = pd.DataFrame({'ATTR':[f], 'IV': [IV(dt, f)]})
    df_iv = df_iv.append(d, ignore_index = True)
df_iv

,ATTR,IV
0,Gender,0.007658
1,Married,0.039175
2,Dependents,0.000575
3,Education,0.023714
4,Self_Employed,0.005920
5,Credit_History,1.731412
6,Property_Area,0.138488
7,APPLICANTINCOME_BIN,0.114022
8,COAPPLICANTINCOME_BIN,0.113386
9,LOANAMOUNT_BIN,0.099628
